In [1]:
import pandas as pd

train = pd.read_pickle("p1_codemix_flat.pkl")
test = pd.read_pickle("p1_codemix_flat_test.pkl")

In [2]:
preds = pd.read_pickle("XLMR_preds_1e-4.pkl")

In [3]:
index = []
for i in range(len(preds)):
        if preds[i]==1:
            index.append(i)

In [4]:
test = test.iloc[index]
train = train.loc[train["label"]==1]

In [5]:
import spacy

NER = spacy.load("en_core_web_sm")

2022-02-18 16:46:14.901980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-18 16:46:14.902019: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
train_distribution = {}
count = 0
for text in train["text"]:
    text1 = NER(text)
    count=count+len(text1)
    for word in text1.ents:
        if word.label_ not in train_distribution:
            train_distribution[word.label_]=1
        else:
            train_distribution[word.label_] = train_distribution[word.label_]+1

In [7]:
test_distribution = {}
count = 0
for text in test["text"]:
    text1 = NER(text)
    count=count+len(text1)
    for word in text1.ents:
        if word.label_ not in test_distribution:
            test_distribution[word.label_]=1
        else:
            test_distribution[word.label_] = test_distribution[word.label_]+1

In [8]:
from collections import OrderedDict

train_distribution_value_sorted = OrderedDict(sorted(train_distribution.items(), key=lambda t: t[1]))

In [9]:
train_keys = list(train_distribution.keys())
test_keys = list(test_distribution.keys())

In [10]:
test_acc_to_train = {}

for key, value in train_distribution_value_sorted.items():
    if key not in test_keys:
        test_acc_to_train[key]=0
    else:
        test_acc_to_train[key] = test_distribution[key]

In [11]:
# test_distribution_value_sorted = OrderedDict(sorted(test_distribution.items(), key=lambda t: t[1]))

In [12]:
# train_distribution_key_sorted = {key: value for key, value in sorted(train_distribution.items())}

# test_distribution_key_sorted = {key: value for key, value in sorted(test_distribution.items())}

In [14]:
import matplotlib.pyplot as plt

plt.plot(train_distribution_value_sorted.keys(), train_distribution_value_sorted.values(), color="r", label = "HOF labelled examples from Train Data") 
plt.plot(test_acc_to_train.keys(), test_acc_to_train.values(), color="blue", label="HOF predicted (using proposed model) Test Data Examples") 
plt.xticks(rotation=90)
plot.legend(loc=2, prop={'size': 6})
plt.xlabel("Named Entities")
plt.ylabel("Frequency")
plt.savefig("namedentity.pdf", bbox_inches="tight")
plt.show()

SyntaxError: invalid syntax (3503599174.py, line 6)

In [ ]:
# train_keys = list(train_distribution.keys())
# test_keys = list(test_distribution.keys())

# for key in train_keys:
#     if key not in test_keys:
#         test_distribution[key]=0

<Figure size 432x288 with 0 Axes>

In [ ]:
import numpy as np
import scipy.stats

x = np.array(list(train_distribution_value_sorted.values()))
y = np.array(list(test_acc_to_train.values()))

In [ ]:
scipy.stats.pearsonr(x, y)
# Correlation, P-Value

(0.9633922250780449, 1.445731346165369e-10)

In [ ]:
scipy.stats.spearmanr(x, y)

SpearmanrResult(correlation=0.9498221118264386, pvalue=1.7255700879492758e-09)

In [ ]:
scipy.stats.kendalltau(x, y)

KendalltauResult(correlation=0.841133386279524, pvalue=1.3884655823390914e-06)